In [5]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import hdbscan
import ast
import itertools

df = pd.read_csv('./results/results_single_prompt_degree_cot_reasoning_factors_list.csv')

/home/ccaesup/miniconda3/envs/LLM_Context_Effects/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
def check_types(lst):
    if not isinstance(lst,str):
        print(f'not a string, the type for {lst} is {type(lst)}')
    else:
        try:
            lst_convert = ast.literal_eval(lst)
            return lst_convert
        except:
            print(f'error in converting {lst}, type is {type(lst)}')
            return []

In [7]:
df['processed_cot_reasoning_2'] = df['processed_cot_reasoning_2'].apply(check_types)

error in converting The text does not provide specific factors considered while comparing the countries., type is <class 'str'>
error in converting The text does not provide specific factors considered while comparing the countries., type is <class 'str'>


In [8]:
type(df['processed_cot_reasoning_2'][0])

list

In [9]:
df['processed_cot_reasoning_1'] = df['processed_cot_reasoning_1'].apply(lambda x: ast.literal_eval(x))
df['processed_cot_reasoning_2'] = df['processed_cot_reasoning_2'].apply(lambda x: ast.literal_eval(x))

ValueError: malformed node or string: ['shared border', 'cultural exchange', 'historical ties', 'economic relationships', 'trade relationships', 'language', 'politics', 'culture', 'indigenous heritage', 'economic orientation', 'income inequality', 'crime rates', 'social issues']

In [11]:
all_topics = list(df['processed_cot_reasoning_1']) + list(df['processed_cot_reasoning_2'])
all_topics =  list(itertools.chain.from_iterable(all_topics))
print(len(all_topics))
set_of_topics = list(set(all_topics))
print(len(set_of_topics))
set_of_topics[:10]

3089
1222


['free market economy',
 'public healthcare system',
 'political turmoil and conflict',
 'geographies',
 'hierarchy of power',
 'shared values and beliefs',
 'geographical',
 'legal structures',
 'atheist ideology',
 'having significant influence on global politics']

In [16]:


# Combine the two columns into a single list of topics

# Load pre-trained Sentence-BERT model
model_name = 'all-mpnet-base-v2'
model = SentenceTransformer(model_name)

# Function to get embeddings
def get_embeddings(text_list):
    return model.encode(text_list)

# Get embeddings for all topics
embeddings = get_embeddings(set_of_topics)

# Use HDBSCAN to cluster topics
clusterer = hdbscan.HDBSCAN(min_cluster_size=6, min_samples=1)
labels = clusterer.fit_predict(embeddings)

# Create a DataFrame to map topics to their cluster labels
# topics_df = pd.DataFrame({'topic': set_of_topics, 'label': labels})

# # Separate the labels back into the original columns
# labels_column1 = topics_df.iloc[:len(df), :]['label'].values
# labels_column2 = topics_df.iloc[len(df):, :]['label'].values

# # Add the labels as new columns to the original DataFrame
# df['label_column1'] = labels_column1
# df['label_column2'] = labels_column2

# # Print the updated DataFrame
# print(df)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/home/ccaesup/miniconda3/envs/LLM_Context_Effects/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
# Map topics to clusters
clustered_topics = {}
for label, topic in zip(labels, set_of_topics):
    if label not in clustered_topics:
        clustered_topics[label] = []
    clustered_topics[label].append(topic)

# Print the clustered topics
for cluster, topics in clustered_topics.items():
    if cluster == -1:
        continue
    print(f"Cluster {cluster}: {topics}")

Cluster 23: ['free market economy', 'economy', 'market economy', 'market-based economies', 'economic', 'economies', 'market-based economy']
Cluster 0: ['public healthcare system', 'healthcare system', 'healthcare', 'universal healthcare', 'approach to healthcare', 'universal healthcare system', 'healthcare policies', 'healthcare systems', 'different healthcare systems']
Cluster 26: ['political turmoil and conflict', 'tensions and conflicts', 'political instability and conflict', 'political upheaval', 'political instability', 'stability of government', 'conflict and instability', 'political stability', 'political changes']
Cluster 41: ['having significant influence on global politics', 'role in global politics', 'role in international affairs', 'influence in global affairs', 'impact on global politics', 'presence in global affairs', 'international influence', 'extent of international involvement', 'role as a global superpower', 'international diplomacy', 'roles in global politics and in

In [4]:
import re

# text = """Cluster 94: ['family ties', 'family connections']
# Cluster 157: ['international alliances', 'responses and alliances', 'strong alliances with other Western countries', 'alliances', 'strong alliance', 'historical alliances', 'alliances and conflicts', 'political alliances', 'alliance history']
# Cluster 76: ['national security strategies', 'belief in the importance of national security', 'emphasis on national security and defense', 'emphasis on national defense', 'commitment to national security', 'commitment to combating terrorism', 'focus on defense and security']
# Cluster 221: ['economic relationship', 'economic relationships', 'economic relations']"""

with open('./results/clusters_all_models_SSD_temp_0.txt', 'r') as f:
    clusters = f.read()

# Split the text into lines
lines = clusters.split('\n')

# Create a list of tuples (cluster_number, cluster_terms)
clusters = []
for line in lines:
    match = re.match(r'Cluster (\d+): \[(.*?)\]', line)
    if match:
        cluster_number = int(match.group(1))
        cluster_terms = match.group(2).strip("'").split("', '")
        clusters.append((cluster_number, cluster_terms))

# Sort the clusters by cluster number
sorted_clusters = sorted(clusters, key=lambda x: x[0])

# Print the sorted clusters
with open('./results/clusters_all_models_SSD_temp_0.txt', 'w') as f:

    for cluster_number, cluster_terms in sorted_clusters:
        print(f"Cluster {cluster_number}: {', '.join(cluster_terms)}")
        f.write(f"Cluster {cluster_number}: {', '.join(cluster_terms)}")

Cluster 0: part of the Holy Roman Empire, part of the Austro-Hungarian Empire
Cluster 1: part of the British Isles, part of the United Kingdom
Cluster 2: sports, horse racing, rugby
Cluster 3: challenges such as drug cartel violence, drug trafficking
Cluster 4: relationship complexity, relationship duration
Cluster 5: existence period, state existence period
Cluster 6: history of Confucianism, history of being influenced by Confucianism
Cluster 7: high standards of living, standards of living, standard of living, high standard of living
Cluster 8: gun control laws, attitudes towards issues such as gun control, gun control policies, gun laws
Cluster 9: labor laws, labor unions, part of the same economic union
Cluster 10: membership in G7, membership in G7 and OECD, membership in the OECD, membership in the G7 and the OECD
Cluster 11: legal and illegal flows of people, legal and illegal flows of services, legal and illegal flows of goods
Cluster 12: mix of urban and rural areas, urban an